In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import cm
from matplotlib.colors import LogNorm
from scipy.ndimage.interpolation import geometric_transform
import statsmodels.api as sm
import re
import os

In [236]:
def find_param(string, param):
    string = string.rpartition('/')[-1]
    string = string.rpartition('.')[0]
    start = string.find(param)
    end = string[start:].find('_')
    if end == -1:
        return string[start+len(param):]
    else:
        return string[start+len(param):start+end]


def createFileList(extension, dirName='.'):
    """Returns a pandas series consisting of all files with the provided
    extension in directory dirName, defaulting to the current working
    directory.
    """
    file_names = [os.path.join(dirName, file) for file in os.listdir(dirName) 
                  if file.rpartition('.')[-1] == extension]
    if len(file_names) == 0:
        print("No '" + extension + "' files found in directory '"
              + dirName + "'.")
        return
    file_names.sort()
    return pd.Series(file_names)


def initializeDataFrame(dirName='.',
                        params=['pf', 'sp', 'lp'],
                        analyses=['global_order']):
    """Generates dataframe of all simcore analysis files, assuming
    file naming convention of containing the substrings listed in the
    params list, followed by the parameter quantity and an underscore.
    This function tabulates these quantities using the file names of any
    bitmaps (final state snapshots) found in snapshotDir. It then looks
    for the corresponding analyses files whose extension is given by the
    substrings found in the analyses list.
    """
    df = None
    params = params + ['reload']
    for analysis in analyses:
        analysis_df = pd.DataFrame(data= createFileList(analysis, dirName), columns=[analysis])
        for param in params:
            analysis_df[param] = analysis_df[analysis].apply(find_param, args=(param,))
        if df is not None:
            df = pd.merge(df, analysis_df, how='outer', on=params)
        else:
            df = analysis_df
    return df[params + analyses]


def initializeDataFrameWithSnapshots(dirName='.',
                        snapshotDir='movies/snapshots',
                        params=['pf','sp','lp'],
                        analyses=['global_order']):
    """Generates dataframe of all simcore analysis files, assuming
    file naming convention of containing the substrings listed in the
    params list, followed by the parameter quantity and an underscore.
    This function tabulates these quantities using the file names of any
    bitmaps (final state snapshots) found in snapshotDir. It then looks
    for the corresponding analyses files whose extension is given by the
    substrings found in the analyses list.
    """
    df=pd.DataFrame(columns=params+['snapshots']+analyses)
    df['snapshots'] = createFileList('bmp',snapshotDir)
    for i in analyses:
        df[i]=createFileList(i,dirName)
    for i in df.index:
        for j in params:
            fname = df.iloc[i]['snapshots']
            fname = fname[-fname[::-1].find('/'):]
            try:
                start = fname.find(j)+len(j)
                end = fname[start:].find('_')
            except:
                print(fname)
            df.iloc[i][j] = float(fname[start:start+end])
    return df


In [21]:
def GetAnalysisGridPlotHandles(df,analyze):
    """Provides the figure and axis handles for a plot grid that are
    determined by the number of unique persistence lengths (lps) and
    U_max's (sps). The dataframe provided must only have one packing
    fraction (pf).
    
    Returns fig, ax, len(sps), len(lps), sps, lps, where the subplot
    dimensions are len(lps) rows and len(sps) columns sps and lps are
    unique, sorted ascending sp and lp entries in dataframe df. 
    """
    assert isinstance(df,pd.DataFrame), (
            "Parameter 'df' must be a pandas dataframe!")
    assert isinstance(analyze,str), (
            "Parameter 'analyze' must be a string!")
    if ('pf' not in df.columns 
        or 'sp' not in df.columns
        or 'lp' not in df.columns):
        print("Dataframe df is missing parameter columns.")
        return None
    if analyze not in df.columns:
        print("Dataframe df is missing "+analyze+" column.")
        return None
    if (len(df['pf'].unique()) != 1):
        print("Dataframe df must only have one unique packing fraction.")
        return None
    is_local = (analyze[:5] == 'local')
    sps = np.sort(df.sp.unique())
    lps = np.sort(df.lp.unique())
    fig_x = len(sps)
    fig_y = len(lps)
    if (fig_x == 0 or fig_y == 0):
        print("Dataframe df is missing entries in sp and/or lp columns.")
        return None
    figscale = (8,6)
    is_not_snapshots=True
    if (analyze == 'snapshots'):
        figscale = (8,8)
        is_not_snapshots=False
    fig,ax = plt.subplots(fig_y, fig_x, 
                          figsize=(figscale[0]*fig_x,figscale[1]*fig_y),
                          sharex=is_local, sharey=is_not_snapshots)
    return fig, ax, fig_x, fig_y, sps, lps

In [22]:
def createSnapshotPlots(df,dirName='.'):
    """Generates a grid of plots displaying snapshot images given by the
    files in the 'snapshots' column of the dataframe, with the images
    arranged to display the effect due to increasing U_max (x-axis) and 
    increasing persistence length/length ratio (y-axis).
    """
    assert isinstance(dirName,str), "'dirName' must be a string!"
    if dirName[-1]=='/':
        dirName=dirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps=GetAnalysisGridPlotHandles(df,'snapshots')
    pf = df.iloc[0].pf
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        a.imshow(mpimg.imread(df.iloc[i].snapshots))
        a.set_yticks([])
        a.set_xticks([])
        a.locator_params(nbins=1)
    for irow,lp in zip(range(fig_y),lps[::-1]):
        ax[irow][0].set_yticks([400])
        ax[irow][0].set_yticklabels([int(lp)],fontsize=50)
        ax[irow][0].tick_params(length=10,width=5)
    for icol,sp in zip(range(fig_x),sps):
        ax[fig_y-1][icol].set_xticks([400])
        ax[fig_y-1][icol].set_xticklabels([int(sp)],fontsize=50)
        ax[fig_y-1][icol].tick_params(length=10,width=5)
    fig.tight_layout(pad=15,h_pad=0,w_pad=-5)
    fig.text(0.5, 0.025, r'$U_{max}/k_B T$', ha='center',fontsize=70)
    fig.text(0.025, 0.5, r'$L_p/L$', va='center',
             rotation='vertical',fontsize=70)
    fig.suptitle("Final state snapshot, "+str(100*pf)
                 +"% pf",fontsize=70,y=0.95)
    fig.savefig(dirName+"/pf"+str(pf)
                +"_final_snapshots.png",dpi=300)

In [338]:
def get_global_order_data(df, saveDirName=".", make_plots=True, params=['pf', 'sp', 'lp'], late_fraction=0.1):
    """Generates two grids of plots displaying time series of the global
    order parameters, including the global polar/nematic order on one
    figure and global spiral number/spiral handedness on a second figure.
    """
    assert isinstance(saveDirName, str), "'dirName' must be a string!"
    analyze = 'global_order'
    if not os.path.exists(saveDirName):
        print("Save directory not found:", saveDirName)
        var = input("Create it? (y/N) ")
        if (var == 'y' or var == 'Y'):
            os.mkdir(saveDirName)
        else:
            raise ValueError("Save directory not found", saveDirName)
    gby = df.groupby(params)
    row_list = []
    for values, group in gby:
        param_values = [i for pair in zip(params, values) for i in pair]
        string_values = str.join('_', ['{}{}' for i in range(len(params))])
        string_values = string_values.format(*param_values)
        display_values = str.join(', ', ['{}={}' for i in range(len(params))])
        display_values = display_values.format(*param_values)
        
        print("Gathering", analyze, "data for parameters", display_values)
        goDF = None
        for file in group[analyze].sort_values():
            if goDF is not None: 
                goDF = goDF.append(GetGlobalOrderDF(file), ignore_index=True)
            else:
                goDF = GetGlobalOrderDF(file)

        if (make_plots):
            fig, ax = plt.subplots(1, 2, figsize=(12, 6))
            goDF.plot(y="nematic_order_mag",color='blue',linewidth=1,ax=ax[0])
            goDF.plot(y="polar_order_mag",color='red',linewidth=1,ax=ax[0])
            ax[0].set_xlabel('Time')
            ax[0].set_ylabel('Orientational order')
            ax[0].legend(['Nematic order', 'Polar order'])
            ax[0].set_ylim(0, 1)
            goDF.plot(y="spiral_order",color='blue',linewidth=1,ax=ax[1])
            goDF.plot(y="signed_spiral_order",color='red',linewidth=1,ax=ax[1])
            ax[1].set_xlabel('Time')
            ax[1].legend(['Spiral order', 'Spiral handedness'])
            ax[1].set_ylabel('Spiral order')
            fig.tight_layout(rect=[0, 0.03, 1, 0.95])
            fig.suptitle("Global order parameters: " + display_values)
            print("Saving", analyze, "plots for parameters", display_values)
            fig.savefig(os.path.join(saveDirName, string_values + "_global_order.png"))
            plt.close(fig)
            
        result_names = ['global_polar', 'global_polar_std', 
                        'global_nematic', 'global_nematic_std',
                        'global_spiral', 'global_spiral_std']
        late_time = int((1 - late_fraction)*goDF.shape[0])
        results = (goDF['polar_order_mag'].iloc[late_time:].mean(), 
                   goDF['polar_order_mag'].iloc[late_time:].std(),
                   goDF['nematic_order_mag'].iloc[late_time:].mean(), 
                   goDF['nematic_order_mag'].iloc[late_time:].std(),
                   goDF['spiral_order'].iloc[late_time:].mean(), 
                   goDF['spiral_order'].iloc[late_time:].std())
        row = {key:value 
               for key, value 
               in (list(zip(params, values)) + list(zip(result_names, results)))}
        row_list.append(row)
    return pd.DataFrame(row_list)
    
def createGlobalOrderPlotsTogether(df,saveDirName="."):
    """Generates two grids of plots displaying time series of the global
    order parameters, including the global polar/nematic order on one
    figure and global spiral number/spiral handedness on a second figure.
    Grid of plots are arranged to display the effect due to increasing
    U_max (x-axis) and increasing persistence length/length ratio (y-axis).
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'global_order'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]   

    # First, we're going to plot nematic and polar global order params
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    pf = df.iloc[0].pf
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        goDF=GetGlobalOrderDF(fname)
        goDF.plot(y="nematic_order_mag",color='blue',linewidth=3,ax=a)
        goDF.plot(y="polar_order_mag",color='red',linewidth=3,ax=a)
        a.set_xlabel('')
        a.legend(['Nematic order','Polar order'],fontsize=20)
        a.tick_params(length=5,width=2.5,labelsize=20)
    for irow in range(fig_y): 
        ax[irow][0].set_ylabel('Order parameter',fontsize=30)
    for icol in range(fig_x): 
        ax[fig_y-1][icol].set_xlabel('Time',fontsize=30)
    fig.tight_layout(pad=15,h_pad=0,w_pad=0)
    fig.text(0.5, 0.05,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.01, 0.5,
             r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Global Polar/Nematic Order, "+str(100*pf)+"% pf",
                 fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_global_order_plots.png",dpi=300)
    
    # Now do the same for spiral order parameters
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        goDF=GetGlobalOrderDF(fname)
        goDF.plot(y="spiral_order",color='blue',linewidth=3,ax=a)
        goDF.plot(y="signed_spiral_order",color='red',linewidth=3,ax=a)
        a.set_xlabel('')
        a.legend(['Spiral order','Spiral handedness'],fontsize=20)
        a.tick_params(length=5,width=2.5,labelsize=20)
    for irow in range(fig_y): 
        ax[irow][0].set_ylabel('Order parameter',fontsize=30)
    for icol in range(fig_x): 
        ax[fig_y-1][icol].set_xlabel('Time',fontsize=30)
    fig.tight_layout(pad=15,h_pad=0,w_pad=0)
    fig.text(0.5, 0.05,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.01, 0.5,
             r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Spiral Order, "+str(100*pf)+"% pf",fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_spiral_order_plots.png",dpi=300)


def CalculateGlobalOrderMagnitudes(df):
    """Given a global order dataframe, calculates magnitude of polar
    order vector and maximum eigenvalue of nematic order tensor.
    """
    df['polar_order_mag'] = np.sqrt(df.polar_order_x**2 
                                    + df.polar_order_y**2 
                                    + df.polar_order_z**2)
    df['nematic_order_mag'] = df.apply(lambda x: maxEig(
        x['nematic_order_xx'],
        x['nematic_order_xy'],
        x['nematic_order_yx'],
        x['nematic_order_yy']), axis=1)

    
def maxEig(xx,xy,yx,yy):
    """Returns the max eigenvalue of 2D matrix with elements xx, xy,
    yx, yy.
    """
    return max(np.linalg.eig(np.array([[xx,xy],[yx,yy]]))[0])

    
def GetGlobalOrderDF(fname):
    """Calculates time series of global orders parameters (polar order
    vector magnitude and maximum eigenvalues of nematic order tensor Q)
    from .global_order file with name 'fname' and returns global order
    dataframe.
    """
    assert isinstance(fname,str), "'fname' must be a string!"
    df = pd.read_csv(fname,delim_whitespace=True,skiprows=1,
                     index_col='time').dropna()
    CalculateGlobalOrderMagnitudes(df)
    return df


In [24]:
def createLocalOrderPlots(df, saveDirName=".", lo_width=20,
                          colorMap=cm.viridis, vlims=(0,3)):
    """Runs createLocalPDFPlot (with the provided vlims),
    createLocalNematicPlot, and createLocalPolarPlot (with default
    vlims between zero and one) in secession, with local order width
    given by lo_width. Each of these function generates a grid of
    plots displaying histograms of the pair distribution function,
    nematic orientation correlation function, and polar order
    correlation functions, respectively. The plots are arranged to
    display the effect due to increasing U_max (x-axis) and increasing
    persistence length/length ratio (y-axis).
    """
    createLocalPDFPlot(df,saveDirName,lo_width,colorMap,vlims)
    createLocalNematicPlot(df,saveDirName,lo_width,colorMap,(0,1))
    createLocalPolarPlot(df,saveDirName,lo_width,colorMap,(0,1))

    
def createLocalPDFPlot(df, saveDirName=".", lo_width=20,
                       colorMap=cm.viridis, vlims=(0,1)):
    """Generates a grid of plots displaying histograms of the pair 
    distribution functions, with the plots arranged to display the
    effect due to increasing U_max (x-axis) and increasing persistence 
    length/length ratio (y-axis).
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'local_pdf'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    fig.tight_layout(pad=15,h_pad=2,w_pad=2)
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    pf = df.iloc[0].pf
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        poDF=pd.read_csv(fname,sep=" ",header=None)
        poDF=poDF.dropna(axis=1)
        im = a.pcolormesh(poDF,vmin=vlims[0], vmax=vlims[1], cmap=colorMap)
        a.set_xlabel('')
        a.tick_params(length=0, width=0, labelsize=20)
        a.locator_params(nbins=0)
    fig.colorbar(im, cax=cbar_ax)
    for irow in range(fig_y): 
        a=ax[irow][0]
        a.set_ylabel('y',fontsize=30)
        a.locator_params(axis='y',nbins=11)
        a.set_yticklabels(-np.round(np.arange(
            -0.5*lo_width,0.5*lo_width+lo_width/10,lo_width/10),1))
        a.tick_params(axis='y',length=5,width=2.5,labelsize=20)
    for icol in range(fig_x): 
        a=ax[fig_y-1][icol]
        a.set_xlabel('x',fontsize=30)
        a.locator_params(axis='x',nbins=11)
        ticks=a.get_xticks()
        a.set_xticks(ticks+min(abs(ticks-100)))
        a.set_xticklabels(map(round,list((ticks-100)*20/200)))
        a.tick_params(axis='x',length=5,width=2.5,labelsize=20)
    cbar_ax.tick_params(length=10,width=5,labelsize=30)
    fig.text(0.45, 0.01,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.0, 0.5, r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Pair Distribution Function, "+str(100*pf)+"% pf",
                 fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_local_pdf_plots.png",dpi=300)
    
    
def createLocalNematicPlot(df, saveDirName=".", lo_width=20,
                           colorMap=cm.viridis, vlims=(0,1)):
    """Generates a grid of plots displaying histograms of the nematic 
    orientation correlation functions, with the plots arranged to display
    the effect due to increasing U_max (x-axis) and increasing persistence 
    length/length ratio (y-axis).
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'local_nematic'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    fig.tight_layout(pad=15,h_pad=2,w_pad=2)
    fig.subplots_adjust(right=0.8)
    pf = df.iloc[0].pf
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        poDF=pd.read_csv(fname,sep=" ",header=None)
        poDF=poDF.dropna(axis=1)
        im = a.pcolormesh(poDF,vmin=vlims[0],vmax=vlims[1],
                    cmap=colorMap)
        a.set_xlabel('')
        a.tick_params(length=0,width=0,labelsize=20)
        a.locator_params(nbins=0)
    fig.colorbar(im, cax=cbar_ax)
    for irow in range(fig_y): 
        a=ax[irow][0]
        a.set_ylabel('y',fontsize=30)
        a.locator_params(axis='y',nbins=11)
        a.set_yticklabels(-np.round(np.arange(
            -0.5*lo_width,0.5*lo_width+lo_width/10,lo_width/10),1))
        a.tick_params(axis='y',length=5,width=2.5,labelsize=20)
    for icol in range(fig_x): 
        a=ax[fig_y-1][icol]
        a.set_xlabel('x',fontsize=30)
        a.locator_params(axis='x',nbins=11)
        ticks=a.get_xticks()
        a.set_xticks(ticks+min(abs(ticks-100)))
        a.set_xticklabels(map(round,list((ticks-100)*20/200)))
        a.tick_params(axis='x',length=5,width=2.5,labelsize=20)
    cbar_ax.tick_params(length=10,width=5,labelsize=30)
    fig.text(0.45, 0.01,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.0, 0.5, r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Nematic Orientational Correlation, "+str(100*pf)+"% pf",
                 fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_local_nematic_plots.png",dpi=300)
    
    
def createLocalPolarPlot(df, saveDirName=".", lo_width=20,
                         colorMap=cm.viridis, vlims=(0,1)):
    """Generates a grid of plots displaying histograms of the polar 
    orientation correlation functions, with the plots arranged to display
    the effect due to increasing U_max (x-axis) and increasing persistence 
    length/length ratio (y-axis).
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'local_polar'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    fig.tight_layout(pad=15,h_pad=2,w_pad=2)
    fig.subplots_adjust(right=0.8)
    pf = df.iloc[0].pf
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        poDF=pd.read_csv(fname,sep=" ",header=None)
        poDF=poDF.dropna(axis=1)
        im = a.pcolormesh(poDF,vmin=vlims[0],vmax=vlims[1],
                    cmap=colorMap)
        a.set_xlabel('')
        a.tick_params(length=0,width=0,labelsize=20)
        a.locator_params(nbins=0)
    fig.colorbar(im, cax=cbar_ax)
    for irow in range(fig_y): 
        a=ax[irow][0]
        a.set_ylabel('y',fontsize=30)
        a.locator_params(axis='y',nbins=11)
        a.set_yticklabels(-np.round(np.arange(
            -0.5*lo_width,0.5*lo_width+lo_width/10,lo_width/10),1))
        a.tick_params(axis='y',length=5,width=2.5,labelsize=20)
    for icol in range(fig_x): 
        a=ax[fig_y-1][icol]
        a.set_xlabel('x',fontsize=30)
        a.locator_params(axis='x',nbins=11)
        ticks=a.get_xticks()
        a.set_xticks(ticks+min(abs(ticks-100)))
        a.set_xticklabels(map(round,list((ticks-100)*20/200)))
        a.tick_params(axis='x',length=5,width=2.5,labelsize=20)
    cbar_ax.tick_params(length=10,width=5,labelsize=30)
    fig.text(0.45, 0.01,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.0, 0.5, r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Polar Orientational Correlation, "+str(100*pf)+"% pf",
                 fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_local_polar_plots.png",dpi=300)
    

In [25]:
def createOrientationCorrPlots(df,saveDirName="."):
    """Generates a grid of plots displaying histograms of the instant
    bond overlap and filament overlap counts as a time series, with the
    plots arranged to display the effect due to increasing U_max (x-axis)
    and increasing persistence length/length ratio (y-axis).
    
    Also generates a second figure of two plots that quantify the
    characteristic orientation decorrelation time as functions of both
    U_max and Lp/L.
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'orientation_corr'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    pf = df.iloc[0].pf
    taus = np.zeros((fig_y,fig_x))
    tau_errors = np.zeros((fig_y,fig_x))
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        ocDF = pd.read_csv(fname,skiprows=1,delim_whitespace=True)
        ocDF = ocDF[:ocDF.time.size-ocDF.time.iloc[-1]-1]
        gb=ocDF.groupby('time')
        x=ocDF.time.unique()
        y=gb.orientation_corr_avg.mean()
        ysem=gb.orientation_corr_sem.mean()
        model=sm.GLM(y, x,
                     family=sm.families.Gaussian(sm.families.links.log))
        fit=model.fit()
        tau=-1.0/fit.params.x1
        taus[irow][icol] = tau
        tau_errors[irow][icol] = tau**2 * fit.bse.x1
        a.errorbar(x,y,yerr=ysem)
        theory = lambda t: np.exp(-t/tau)
        a.plot(x,theory(x),'r')
        a.set_ylabel(r"$\langle u(0)\cdot u(t)\rangle$",fontsize=18)
        a.set_xlabel("Time (t)",fontsize=18)
        a.legend([r"Fit, $\tau=%2.2f$" % tau,'Simulation'],fontsize=16)
        a.grid(True,linestyle='--')
    for irow in range(fig_y): 
        ax[irow][0].set_ylabel(
            r'$\langle u(t)\cdot u(t+\tau) \rangle$',fontsize=30)
    for icol in range(fig_x): 
        ax[fig_y-1][icol].set_xlabel('Time',fontsize=30)
    fig.tight_layout(pad=15,h_pad=0,w_pad=0)
    fig.text(0.5, 0.05,
             r'$\Longrightarrow \epsilon \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.01, 0.5,
             r'$\Longrightarrow l \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Orientation autocorrelation, "+r'$\phi$ = '+str(pf),fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_orientation_corr_plots.png",dpi=300)
    fig,ax=plt.subplots(1,2,figsize=(16,6))
    for i in range(taus.shape[1]):
        ax[0].errorbar(lps[::-1],taus[:,i],yerr=tau_errors[:,i],
                       capsize=2,capthick=2)
    for i in range(taus.shape[0]):
        ax[1].errorbar(sps,taus[i,:],yerr=tau_errors[i,:],
                       capsize=2,capthick=2)
    for a in ax:
        #a.grid(True,linestyle='--')
        a.set_ylabel(r'$\tau_\phi$',fontsize=30)
        a.tick_params(labelsize=16)
    leg = ax[0].legend([str(i) for i in [1.40, 1.86, 2.33]],title=r'$\epsilon^*$',fontsize=20)
    plt.setp(leg.get_title(),fontsize='25')
    ax[0].set_xlabel(r'$\kappa$',fontsize=30)
    ax[1].set_xlabel(r'$\epsilon^*$',fontsize=30)
    ax[1].legend([r'$\kappa=$'+str(i) for i in lps],fontsize=20)
    fig.suptitle(r'Orientation autocorrelation lifetime $\tau_\phi$, $\phi$ = '+str(pf),
                 fontsize=30)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_orientation_corr_graph_plots.png",dpi=300)


In [26]:
def createPolarOrderPlots(df,saveDirName=".",contact_cut=10,
                          colorMap=cm.viridis,vlims=(1e-7,1e-1)):
    """Generates a grid of plots displaying histograms of the local
    polar order as a function of contact number, with the plots arranged
    to display the effect due to increasing U_max (x-axis) and increasing
    persistence length/length ratio (y-axis). The parameter contact_cut
    is used to renormalize the x-axis ticklabels to be in the range
    (0,contact_cut).
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'polar_order'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]
    fig.tight_layout(pad=15,h_pad=0,w_pad=0)
    fig.subplots_adjust(right=0.8)
    pf = df.iloc[0].pf
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        poDF=pd.read_csv(fname,skiprows=2,delim_whitespace=True,
                         header=None)
        poDF=poDF.dropna(axis=1)
        data = poDF.replace(0,1)
        data=data/data.sum().sum()
        min_data = data.min().min()
        if (min_data == 0):
            min_data = 1
        max_data = data.max().max()
        log_norm = LogNorm(vmin=min_data, vmax=max_data)
        cbar_ticks = [10**i for i in 
                      range(int(np.floor(np.log10(min_data))),
                            1 + int(np.ceil(np.log10(max_data))))]
        im = a.pcolormesh(data, vmin=vlims[0],vmax=vlims[1],norm=log_norm,
                    cmap=cm.viridis)
        a.set_xlabel('')
        a.tick_params(length=0,width=0,labelsize=20)
        a.set_xticklabels([])
        a.set_yticklabels([])
    fig.colorbar(im, cax=cbar_ax, ticks=cbar_ticks)
    for irow in range(fig_y): 
        a=ax[irow][0]
        a.set_ylabel(r'Local Polar Order, $p_i$',fontsize=30)
        a.locator_params(axis='y',nbins=21)
        a.tick_params(axis='y',length=5,width=2.5,labelsize=20)
        ticks=a.get_yticks()
        a.set_yticks(ticks+min(abs(ticks-25)))
        k=list(-(ticks/50-1))
        a.set_yticklabels(np.round(k-min([abs(i) for i in k]),2))
    for icol in range(fig_x): 
        a=ax[fig_y-1][icol]
        a.set_xlabel(r'Contact Number, $c_i$',fontsize=30)
        a.locator_params(axis='x',nbins=21)
        a.tick_params(axis='x',length=5,width=2.5,labelsize=20)
        ticks=a.get_xticks()
        a.set_xticklabels(
            np.round(list((ticks-min(ticks))*contact_cut/100),1))
    cbar_ax.tick_params(length=10,width=5,labelsize=30)
    fig.text(0.45, 0.01,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.0, 0.5, r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Local Polar Order, "+str(100*pf)+"% pf",fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_polar_order_plots.png",dpi=300)



def createPolarOrderPlotsTogether(df,saveDirName=".",contact_cut=10,
                          colorMap=cm.viridis,vlims=(1e-7,1e-1)):
    """Generates a grid of plots displaying histograms of the local
    polar order as a function of contact number, with the plots arranged
    to display the effect due to increasing U_max (x-axis) and increasing
    persistence length/length ratio (y-axis). The parameter contact_cut
    is used to renormalize the x-axis ticklabels to be in the range
    (0,contact_cut).
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'polar_order'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    fig.tight_layout(pad=15,h_pad=0,w_pad=0)
    fig.subplots_adjust(right=0.8)
    pf = df.iloc[0].pf
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        poDF=pd.read_csv(fname,skiprows=2,delim_whitespace=True,
                         header=None)
        poDF=poDF.dropna(axis=1)
        data = poDF.replace(0,1)
        data=data/data.sum().sum()
        min_data = data.min().min()
        if (min_data == 0):
            min_data = 1
        max_data = data.max().max()
        log_norm = LogNorm(vmin=min_data, vmax=max_data)
        cbar_ticks = [10**i for i in 
                      range(int(np.floor(np.log10(min_data))),
                            1 + int(np.ceil(np.log10(max_data))))]
        im = a.pcolormesh(data, vmin=vlims[0],vmax=vlims[1],norm=log_norm,
                    cmap=cm.viridis)
        a.set_xlabel('')
        a.tick_params(length=0,width=0,labelsize=20)
        a.set_xticklabels([])
        a.set_yticklabels([])
    fig.colorbar(im, cax=cbar_ax, ticks=cbar_ticks)
    for irow in range(fig_y): 
        a=ax[irow][0]
        a.set_ylabel(r'Local Polar Order, $p_i$',fontsize=30)
        a.locator_params(axis='y',nbins=21)
        a.tick_params(axis='y',length=5,width=2.5,labelsize=20)
        ticks=a.get_yticks()
        a.set_yticks(ticks+min(abs(ticks-25)))
        k=list(-(ticks/50-1))
        a.set_yticklabels(np.round(k-min([abs(i) for i in k]),2))
    for icol in range(fig_x): 
        a=ax[fig_y-1][icol]
        a.set_xlabel(r'Contact Number, $c_i$',fontsize=30)
        a.locator_params(axis='x',nbins=21)
        a.tick_params(axis='x',length=5,width=2.5,labelsize=20)
        ticks=a.get_xticks()
        a.set_xticklabels(
            np.round(list((ticks-min(ticks))*contact_cut/100),1))
    cbar_ax.tick_params(length=10,width=5,labelsize=30)
    fig.text(0.45, 0.01,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.0, 0.5, r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Local Polar Order, "+str(100*pf)+"% pf",fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_polar_order_plots.png",dpi=300)


In [27]:
def createOverlapPlots(df,saveDirName="."):
    """Generates a grid of plots displaying histograms of the instant
    bond overlap and filament overlap counts as a time series, with the
    plots arranged to display the effect due to increasing U_max (x-axis)
    and increasing persistence length/length ratio (y-axis).
    
    Also generates a second plot that quantifies the overlap initiation
    rate as a function of U_max and with Lp/L as multiple plot lines.
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'overlaps'
    if saveDirName[-1]=='/':
        saveDirName=saveDirName[:-1]
    
    # Generate time series of instantaneous bond and filament overlaps.
    # Also, use the time series data of filament overlap initiations
    # (which is very linear) to determine their rate.
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    pf = df.iloc[0].pf
    init_rates=np.empty((fig_y,fig_x),dtype=object)
    rate_errors=np.empty((fig_y,fig_x),dtype=object)
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        goDF=pd.read_csv(fname,delim_whitespace=True)
        goDF['n_tangled'] = (goDF['n_total_crossings_init'] 
                             - goDF['n_total_crossings_complete'])
        goDF.plot(x='time',y="n_instant_bond_overlaps",
                  color='blue',linewidth=3,ax=a)
        goDF.plot(x='time',y="n_tangled",color='red',linewidth=3,ax=a)
        begin=int(goDF['time'].size/10)
        model=sm.OLS(goDF['n_total_crossings_init'].iloc[begin:],
                     sm.add_constant(goDF['time'].iloc[begin:]))
        model=model.fit()
        init_rates[irow][icol]=model.params[1]
        rate_errors[irow][icol]=model.bse[1]
        a.set_xlabel('')
        a.legend(['Bond overlaps','Filament overlaps'],fontsize=20)
        a.tick_params(length=5,width=2.5,labelsize=20)
        a.ticklabel_format(style='sci', axis='y', scilimits=(0,0),
                           useMathText=True)
        a.yaxis.get_offset_text().set_fontsize(20)
    for irow in range(fig_y): 
        ax[irow][0].set_ylabel('Overlap number',fontsize=30)
    for icol in range(fig_x): 
        ax[fig_y-1][icol].set_xlabel('Time',fontsize=30)
    fig.tight_layout(pad=15,h_pad=0,w_pad=0)
    fig.text(0.5, 0.05,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.01, 0.5, r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Filament overlap events, "+str(100*pf)+"% pf",fontsize=70,
                 y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)+"_overlap_plots.png",dpi=300)
    
    # Now plot the overlap initiation rates as a funciton of U_max,
    # with each Lp/L on its own line.
    fig,ax=plt.subplots(1,1,figsize=(8,6))
    for i in range(init_rates.shape[0]):
        ax.errorbar(sps,init_rates[i,:],yerr=rate_errors[i,:],capsize=2,
                    capthick=2)
    ax.grid(True,linestyle='--')
    ax.set_ylabel(r'$f_o$',fontsize=30)
    ax.tick_params(labelsize=16)
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0),
                        useMathText=True)
    ax.yaxis.get_offset_text().set_fontsize(16)
    ax.set_xlabel(r'$U_{max}/k_B T$',fontsize=30)
    ax.legend([r'$L_p/L=$'+str(i) for i in lps[::-1]],fontsize=20)
    fig.suptitle(r'Overlap initation rate $f_o$, '+str(100*pf)
                 +'% pf',fontsize=25)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_overlap_rate_plot.png",dpi=300)


In [28]:
def foldTimeSeries(ts):
    """This function takes a symmetric, even time series and returns the
    averaged time series that is 1/2 the original length, effectively
    folding the original time series along the time-axis and averaging
    the results.
    """
    assert isinstance(ts,(np.ndarray,pd.Series)), (
        "ts must be of pandas Series or numpy ndarray type")
    if isinstance(ts,pd.Series):
        return 0.5 * (ts[:len(ts)//2][::-1].reset_index(drop=True)
                      + ts[len(ts)//2:].reset_index(drop=True))
    elif isinstance(ts, np.ndarray):
        return 0.5 * (ts[:len(ts)//2][::-1] + ts[len(ts)//2:])
    else:
        print("Time series datatype unrecognized in foldTimeSeries!")
        
def recenteringFunction(y_mx, y_2nd, y_3rd, x_mx, x_2nd, x_3rd):
    """This function takes in the a local maximum of a coarse-grained
    time series function (y_mx) as well as its two nearest-neighbor
    data points (y_2nd, y_3rd) as well as their corresponding values
    on the x-axis (x_mx, x_2nd, x_3rd). The function then attempts to
    estimate the likely position of the "true" maximum of the function
    represented by the data w.r.t the x-axis.
    
    E.g. if y_mx ~= y_2nd, then the location of the true maximum
    x_max_true is likely avg(x_mx, x_2nd). If y_2nd ~= y_3rd, then
    x_max_true ~= x_mx.
    """
    assert (y_mx >= y_2nd and y_mx >= y_3rd), (
        "y_mx must be greater than or equal to y_2nd and y_3rd")
    # Ensure that y_2nd >= y_3rd
    if (y_3rd > y_2nd):
        temp = y_2nd
        y_2nd = y_3rd
        y_3rd = temp
        temp = x_2nd
        x_2nd = x_3rd
        x_3rd = temp
    # If it is the case that y_mx - y_3rd is zero, then we must
    # also have the case that y_mx == y_2nd == y_3rd
    if (y_mx - y_3rd == 0):
        return x_mx
    weight = (y_mx-y_2nd) / (y_mx-y_3rd)
    return (1-weight)*(x_mx+x_2nd)/2 + weight*x_mx

In [29]:
def createLocalPDFPlot1D(df, saveDirName=".", lo_width=20,
                       colorMap=cm.viridis, vlims=(0,1)):
    """Generates a grid of plots displaying histograms of the pair 
    distribution functions, with the plots arranged to display the
    effect due to increasing U_max (x-axis) and increasing persistence 
    length/length ratio (y-axis).
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'local_pdf'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    fig.tight_layout(pad=15,h_pad=2,w_pad=2)
    pf = df.iloc[0].pf
    nums = np.zeros((len(lps),len(sps)))
    for i in df.index:
        icol = np.where(sps==df.iloc[i].sp)[0][0]
        irow = fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname = df.iloc[i][analyze]
        poDF = pd.read_csv(fname,sep=" ",header=None)
        poDF = poDF.dropna(axis=1)
        fft_data = np.abs(np.fft.fftshift(np.fft.fft2(poDF)))
        midpoint = poDF.shape[0]//2
        y_mid = poDF.iloc[midpoint]
        fft_midpoint = fft_data.shape[0]//2
        y_norm = 1 + 1/250*fft_data[fft_midpoint]
        y_mid = y_norm
        nums[irow][icol] = np.max(y_norm)
        y_avg = poDF.mean(axis=0)
        y = y_mid # choose not to average along y-axis
        x = np.linspace(-10, 10, len(y))
        y_fold = foldTimeSeries(y)
        x_fold = np.linspace(0, 10, len(y_fold))
        a.plot(x_fold, y_fold, linewidth=3)
        a.grid(True, linestyle='dashed')
    for irow in range(fig_y): 
        a=ax[irow][0]
        a.set_ylabel(r'$g(r_{\perp})$',fontsize=30)
    for icol in range(fig_x): 
        a=ax[fig_y-1][icol]
        a.set_xlabel(r'$r_{\perp}$',fontsize=30)
    fig.text(0.45, 0.01,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.0, 0.5, r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Pair Distribution Function 1D, "+str(100*pf)+"% pf",
                 fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_local_pdf_1d_plots.png",dpi=300)
    fig, ax = plt.subplots(1,1)
    for i in range(nums.shape[0]):
        ax.plot(sps,nums[i],'-o')
    ax.grid(True)
    ax.legend([100,50,20])
    ax.set_xlabel(r'$U_{max}/k_B T$',fontsize=15)
    ax.set_ylabel(r'$\hat{g}_{max}(q)$',fontsize=15)
    ax.set_title("Structural order factor")
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_local_pdf_structure_plots.png",dpi=300)

In [30]:
def createStructurePlots(df, saveDirName=".", colorMap=cm.viridis):
    """Generates a grid of structure factors from time averaged 
    density FFTs.
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'structure'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    fig.tight_layout(pad=15,h_pad=0,w_pad=0)
    fig.subplots_adjust(right=0.8)
    pf = df.iloc[0].pf
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname = df.iloc[i][analyze]
        poDF = pd.read_csv(fname, delim_whitespace=True, header=None)
        data = np.fft.fftshift(poDF)
        min_data = data.min().min()
        if (min_data == 0):
            min_data = 1e-16
        max_data = data.max().max()
        log_norm = LogNorm(vmin=min_data, vmax=max_data)
        cbar_ticks = [10**i for i in 
                      range(int(np.floor(np.log10(min_data))),
                            1 + int(np.ceil(np.log10(max_data))))]
        im = a.pcolormesh(data, norm=log_norm, cmap=cm.viridis)
        #fig.colorbar(im, cax=cbar_ax, ticks=cbar_ticks)
        a.set_xlabel('')
        a.tick_params(length=0,width=0,labelsize=20)
        a.set_xticklabels([])
        a.set_yticklabels([])
    fig.colorbar(im, cax=cbar_ax, ticks=cbar_ticks)
    for irow in range(fig_y): 
        a=ax[irow][0]
        a.set_ylabel(r'$k_y$',fontsize=30)
        a.locator_params(axis='y',nbins=11)
        a.tick_params(axis='y',length=5,width=2.5,labelsize=20)
        ticks=a.get_yticks()
        a.set_yticklabels(['{0:2.2f}'.format(2*np.pi*(x-500)/1000) 
                           for x in np.linspace(0,1000,len(ticks))])
    for icol in range(fig_x): 
        a=ax[fig_y-1][icol]
        a.set_xlabel(r'$k_x$',fontsize=30)
        a.locator_params(axis='x',nbins=11)
        a.tick_params(axis='x',length=5,width=2.5,labelsize=20)
        ticks=a.get_xticks()
        a.set_xticklabels(['{0:2.2f}'.format(2*np.pi*(x-500)/1000) 
                           for x in np.linspace(0,1000,len(ticks))])
    cbar_ax.tick_params(length=10,width=5,labelsize=30)
    fig.text(0.45, 0.01,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.0, 0.5, r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Structure Factor, "+str(100*pf)+"% pf",
                 fontsize=70, y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_structure_factor_plots.png",dpi=300)


In [333]:
# Requires time step due to output file header not providing the data...
def createPolarOrderAvgPlots(df, time_step, saveDirName=".", params=['pf', 'sp', 'lp'], 
                             make_plots=True, late_fraction=0.1, rolling_window=20):
    """Generates two grids of plots displaying time series of the global
    order parameters, including the global polar/nematic order on one
    figure and global spiral number/spiral handedness on a second figure.
    """
    assert isinstance(saveDirName, str), "'dirName' must be a string!"
    analyze = 'polar_order_avg'
    if not os.path.exists(saveDirName):
        print("Save directory not found:", saveDirName)
        var = input("Create it? (y/N) ")
        if (var == 'y' or var == 'Y'):
            os.mkdir(saveDirName)
        else:
            raise ValueError("Save directory not found", saveDirName)
    gby = df.groupby(params)
    row_list = []
    for values, group in gby:
        param_values = [i for pair in zip(params, values) for i in pair]
        string_values = str.join('_', ['{}{}' for i in range(len(params))])
        string_values = string_values.format(*param_values)
        display_values = str.join(', ', ['{}={}' for i in range(len(params))])
        display_values = display_values.format(*param_values)
        
        goDF = None
        for file in group[analyze].sort_values():
            if goDF is not None: 
                goDF = goDF.append(pd.read_csv(file, skiprows=1, delim_whitespace=True), 
                                   ignore_index=True)
            else:
                goDF = pd.read_csv(file, skiprows=1, delim_whitespace=True)
        goDF['time'] = goDF.index * time_step
        if make_plots:
            fig, ax = plt.subplots(1, 1, figsize=(6, 6))
            plotDF = goDF.rolling(rolling_window).mean().dropna()
            plotDF.plot(x="time", y="avg_polar_order", color='red',
                      linewidth=1, ax=ax, label=r'$\langle p_i \rangle$')
            plotDF.plot(x="time", y="avg_contact_number", color='blue',
                      linewidth=1, ax=ax, label=r'$\langle c_i \rangle$')
            ax.set_xlabel('Time')
            ax.set_ylabel('Order parameter')
            ax.set_title('Average local polar order: ' + display_values)
            ax.legend(loc='best')
            fig.tight_layout(rect=[0, 0.03, 1, 0.95])
            print("Saving", analyze, "plots for parameters", display_values)
            fig.savefig(os.path.join(saveDirName, string_values + "_local_polar_order_avg.png"))
            plt.close(fig)
        result_names = ['avg_polar_order', 'avg_polar_order_std', 
                        'avg_contact_number', 'avg_contact_number_std']
        late_time = 1 - late_fraction
        results = (goDF['avg_polar_order'].iloc[int(late_time*goDF.shape[0]):].mean(), 
                   goDF['avg_polar_order'].iloc[int(late_time*goDF.shape[0]):].std(),
                   goDF['avg_contact_number'].iloc[int(late_time*goDF.shape[0]):].mean(), 
                   goDF['avg_contact_number'].iloc[int(late_time*goDF.shape[0]):].std())
        row = {key:value 
               for key, value 
               in (list(zip(params, values)) + list(zip(result_names, results)))}
        row_list.append(row)
    return pd.DataFrame(row_list)

def createPolarOrderAvgPlotsTogether(df,saveDirName="."):
    """Generates a grids of plots displaying time series of the average
    local polar order.
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'polar_order_avg'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]   

    # First, we're going to plot nematic and polar global order params
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    pf = df.iloc[0].pf
    means = np.zeros((fig_y, fig_x))
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        goDF=pd.read_csv(fname, skiprows=1, delim_whitespace=True)
        goDF.plot(x="time", y="avg_polar_order", color='blue',
                  linewidth=3, ax=a)
        late_time = int(2/3 * len(goDF['avg_polar_order']))
        late_sim_mean = goDF['avg_polar_order'].iloc[late_time:].mean()
        means[irow][icol] = late_sim_mean
        a.set_xlabel('')
        a.tick_params(length=5, width=2.5, labelsize=20)
        a.grid(True, linestyle='dashed')
    for irow in range(fig_y): 
        ax[irow][0].set_ylabel(r'$\langle$ Local polar order $\rangle$',
                               fontsize=30)
    for icol in range(fig_x): 
        ax[fig_y-1][icol].set_xlabel('Time', fontsize=30)
    fig.tight_layout(pad=15, h_pad=0, w_pad=0)
    fig.text(0.5, 0.05,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center', fontsize=70)
    fig.text(0.01, 0.5,
             r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical', fontsize=70)
    fig.suptitle("Average Local Polar Order, "+str(100*pf)+"% pf",
                 fontsize=70, y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_polar_order_avg_plots.png", dpi=300)
    
    # Now create aggregate data plot for the late sim-time means
    fig, ax = plt.subplots(1,1)
    for i in range(means.shape[0]):
        ax.plot(sps, means[i], '-o')
    ax.grid(True, linestyle='dashed')
    ax.legend([r'$L_p/L=$'+str(i) for i in lps[::-1]],fontsize=15)
    ax.set_xlabel(r'$U_{max}/k_B T$',fontsize=15)
    ax.set_ylabel(r'$\langle p_i \rangle$',fontsize=15)
    ax.set_title("Average local polar order")
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_polar_order_avg_aggregate.png",dpi=300)
    return sps,means

In [32]:
def ToPolar(img, order=3, mode='constant', cval=0):
    max_radius = 0.5*np.linalg.norm( img.shape )
    def transform(coords):
        theta = 2.0*np.pi*coords[1] / (img.shape[1] - 1.)
        radius = max_radius * coords[0] / img.shape[0]
        i = 0.5*img.shape[0] - radius*np.sin(theta)
        j = radius*np.cos(theta) + 0.5*img.shape[1]
        return i,j
    polar = geometric_transform(img, transform, order=order, mode=mode, 
                                cval=cval, prefilter=True)
    return np.log(pd.DataFrame(
        np.abs(polar)[:-1]).replace(0, np.nan).mean(axis=1, skipna=True))

def createStrutureRadialAvgPlots(df,saveDirName="."):
    """Generates a grids of plots displaying time series of the
    radially-averaged structure factor as well as the aggregation of
    the second peak maxima.
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'structure'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    pf = df.iloc[0].pf
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname = df.iloc[i][analyze]
        poDF = pd.read_csv(fname, delim_whitespace=True, header=None)
        data = np.fft.fftshift(poDF)
        pol = ToPolar(data)
        if i == 0:
            pols = np.zeros((fig_y, fig_x, len(pol)))
            pol_max_vals = np.zeros((fig_y, fig_x, 2))
            mid = len(pol)//2
        pols[irow][icol] = pol
        max_ind = np.argmax(pol[mid:])
        pol_max_vals[irow][icol][0] = max_ind
        pol_max_vals[irow][icol][1] = pol[max_ind]
        a.grid(True, linestyle='dashed')
        a.plot(pol)

    pol_idx = np.where(pol_max_vals==np.max(pol_max_vals[:,:,1]))
    max_idx = int(pol_max_vals[pol_idx[0],pol_idx[1],0][0])

    for irow in range(fig_y):
        for icol in range(fig_x):
            search_range = int(0.05*mid)
            pol_max_vals[irow][icol][0] = (max_idx-search_range
                                           +np.argmax(
                pols[irow][icol][max_idx-search_range:max_idx+
                                 search_range])
            )
            pol_max_vals[irow][icol][1] = (
                pols[irow][icol][int(pol_max_vals[irow][icol][0])]
            )
            a = ax[irow][icol]
            ymin, ymax = a.get_ylim()
            a.vlines(int(pol_max_vals[irow][icol][0]), ymin, ymax, 
                     colors='red', linestyle='dashed')
            a.text(0.85, 0.90,
                   '{0:.2f}'.format(pol_max_vals[irow][icol][1]),
                   verticalalignment='bottom', 
                   horizontalalignment='left',
                   transform=a.transAxes,
                   color='red', fontsize=15)
            a.set_xlabel('')
            a.tick_params(length=5,width=2.5,labelsize=20)
            a.grid(True, linestyle='dashed')

    for irow in range(fig_y): 
        ax[irow][0].set_ylabel(r'$\langle S(r) \rangle$',fontsize=20)
    for icol in range(fig_x): 
        ax[fig_y-1][icol].set_xlabel(r'$r$',fontsize=30)
    fig.tight_layout(pad=15,h_pad=0,w_pad=0)
    fig.text(0.5, 0.05,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.01, 0.5,
             r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle('Radially-averaged structure factor, '+
                 r'$\langle S(r) \rangle$, '+str(100*pf)+"% pf",
                 fontsize=70,y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_structure_radial_avg_plots.png",dpi=300)
    
    # Now plot the second peak maxima on one plot
    nums = np.zeros((fig_y,fig_x))
    min_val = 1e6
    for irow in range(fig_y):
        for icol in range(fig_x):
            nums[irow][icol] = pol_max_vals[irow][icol][1]
            if nums[irow][icol] < min_val:
                min_val = nums[irow][icol]
    for irow in range(fig_y):
        for icol in range(fig_x):
            nums[irow][icol] -= min_val
    fig, ax = plt.subplots(1,1)
    for i in range(nums.shape[0]):
        ax.plot(sps,nums[i],'-o')
    ax.grid(True)
    ax.legend([100,50,20])
    ax.set_xlabel(r'$U_{max}/k_B T$',fontsize=15)
    ax.set_ylabel(r'$\log(\langle S(r) \rangle)$, second peak',fontsize=15)
    ax.set_title("Structural order factor")
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_structure_radial_avg_aggregate.png",dpi=300)
    return sps,nums

In [33]:
def createAnalysisPlots(df,saveDirName='.'):
    """This function runs the following functions in secession:
    createSnapshotPlots, createGlobalOrderPlots, createLocalOrderPLots,
    createPolarOrderPlots, createOrientationCorrPlots, createOverlapPlots.
    In addition, only function defaults are called, except for the passed
    dataframe and the directory where all the plots are saved.
    """
    createSnapshotPlots(df,saveDirName)
    createGlobalOrderPlots(df,saveDirName)
    createLocalOrderPlots(df,saveDirName)
    createPolarOrderPlots(df,saveDirName)
    createOrientationCorrPlots(df,saveDirName)
    createOverlapPlots(df,saveDirName)

In [244]:
df = initializeDataFrame('order_params/',
                    params=['pf', 'sp', 'lp'],
                    analyses=['global_order',
                              'polar_order',
                              'polar_order_avg'])
df.head()

,pf,sp,lp,reload,global_order,polar_order,polar_order_avg
0,0.1,005,0005,001,order_params/activeff_pf0.1_sp005_lp0005_reloa...,order_params/activeff_pf0.1_sp005_lp0005_reloa...,order_params/activeff_pf0.1_sp005_lp0005_reloa...
1,0.1,005,0005,002,order_params/activeff_pf0.1_sp005_lp0005_reloa...,order_params/activeff_pf0.1_sp005_lp0005_reloa...,order_params/activeff_pf0.1_sp005_lp0005_reloa...
2,0.1,005,0010,001,order_params/activeff_pf0.1_sp005_lp0010_reloa...,order_params/activeff_pf0.1_sp005_lp0010_reloa...,order_params/activeff_pf0.1_sp005_lp0010_reloa...
3,0.1,005,0010,002,order_params/activeff_pf0.1_sp005_lp0010_reloa...,order_params/activeff_pf0.1_sp005_lp0010_reloa...,order_params/activeff_pf0.1_sp005_lp0010_reloa...
4,0.1,005,0020,001,order_params/activeff_pf0.1_sp005_lp0020_reloa...,order_params/activeff_pf0.1_sp005_lp0020_reloa...,order_params/activeff_pf0.1_sp005_lp0020_reloa...


In [336]:
go_results = createGlobalOrderPlots(df, 'order_params/plots')

Saving plots for parameters pf=0.1, sp=005, lp=0005
Saving plots for parameters pf=0.1, sp=005, lp=0010
Saving plots for parameters pf=0.1, sp=005, lp=0020
Saving plots for parameters pf=0.1, sp=005, lp=0030
Saving plots for parameters pf=0.1, sp=005, lp=0040
Saving plots for parameters pf=0.1, sp=005, lp=0050
Saving plots for parameters pf=0.1, sp=005, lp=0060
Saving plots for parameters pf=0.1, sp=005, lp=0070
Saving plots for parameters pf=0.1, sp=005, lp=0080
Saving plots for parameters pf=0.1, sp=005, lp=0090
Saving plots for parameters pf=0.1, sp=005, lp=0100
Saving plots for parameters pf=0.1, sp=005, lp=0150
Saving plots for parameters pf=0.1, sp=005, lp=0200
Saving plots for parameters pf=0.1, sp=005, lp=0300
Saving plots for parameters pf=0.1, sp=005, lp=0500
Saving plots for parameters pf=0.1, sp=005, lp=1000
Saving plots for parameters pf=0.1, sp=010, lp=0005
Saving plots for parameters pf=0.1, sp=010, lp=0010
Saving plots for parameters pf=0.1, sp=010, lp=0020
Saving plots

Saving plots for parameters pf=0.1, sp=090, lp=0500
Saving plots for parameters pf=0.1, sp=090, lp=1000
Saving plots for parameters pf=0.1, sp=100, lp=0005
Saving plots for parameters pf=0.1, sp=100, lp=0010
Saving plots for parameters pf=0.1, sp=100, lp=0020
Saving plots for parameters pf=0.1, sp=100, lp=0030
Saving plots for parameters pf=0.1, sp=100, lp=0040
Saving plots for parameters pf=0.1, sp=100, lp=0050
Saving plots for parameters pf=0.1, sp=100, lp=0060
Saving plots for parameters pf=0.1, sp=100, lp=0070
Saving plots for parameters pf=0.1, sp=100, lp=0080
Saving plots for parameters pf=0.1, sp=100, lp=0090
Saving plots for parameters pf=0.1, sp=100, lp=0100
Saving plots for parameters pf=0.1, sp=100, lp=0150
Saving plots for parameters pf=0.1, sp=100, lp=0200
Saving plots for parameters pf=0.1, sp=100, lp=0300
Saving plots for parameters pf=0.1, sp=100, lp=0500
Saving plots for parameters pf=0.1, sp=100, lp=1000
Saving plots for parameters pf=0.1, sp=200, lp=0005
Saving plots

In [337]:
po_results = createPolarOrderAvgPlots(df, 'order_params/plots')

Saving plots for parameters pf=0.1, sp=005, lp=0005
Saving plots for parameters pf=0.1, sp=005, lp=0010
Saving plots for parameters pf=0.1, sp=005, lp=0020
Saving plots for parameters pf=0.1, sp=005, lp=0030
Saving plots for parameters pf=0.1, sp=005, lp=0040
Saving plots for parameters pf=0.1, sp=005, lp=0050
Saving plots for parameters pf=0.1, sp=005, lp=0060
Saving plots for parameters pf=0.1, sp=005, lp=0070
Saving plots for parameters pf=0.1, sp=005, lp=0080
Saving plots for parameters pf=0.1, sp=005, lp=0090
Saving plots for parameters pf=0.1, sp=005, lp=0100
Saving plots for parameters pf=0.1, sp=005, lp=0150
Saving plots for parameters pf=0.1, sp=005, lp=0200
Saving plots for parameters pf=0.1, sp=005, lp=0300
Saving plots for parameters pf=0.1, sp=005, lp=0500
Saving plots for parameters pf=0.1, sp=005, lp=1000
Saving plots for parameters pf=0.1, sp=010, lp=0005
Saving plots for parameters pf=0.1, sp=010, lp=0010
Saving plots for parameters pf=0.1, sp=010, lp=0020
Saving plots

Saving plots for parameters pf=0.1, sp=090, lp=0500
Saving plots for parameters pf=0.1, sp=090, lp=1000
Saving plots for parameters pf=0.1, sp=100, lp=0005
Saving plots for parameters pf=0.1, sp=100, lp=0010
Saving plots for parameters pf=0.1, sp=100, lp=0020
Saving plots for parameters pf=0.1, sp=100, lp=0030
Saving plots for parameters pf=0.1, sp=100, lp=0040
Saving plots for parameters pf=0.1, sp=100, lp=0050
Saving plots for parameters pf=0.1, sp=100, lp=0060
Saving plots for parameters pf=0.1, sp=100, lp=0070
Saving plots for parameters pf=0.1, sp=100, lp=0080
Saving plots for parameters pf=0.1, sp=100, lp=0090
Saving plots for parameters pf=0.1, sp=100, lp=0100
Saving plots for parameters pf=0.1, sp=100, lp=0150
Saving plots for parameters pf=0.1, sp=100, lp=0200
Saving plots for parameters pf=0.1, sp=100, lp=0300
Saving plots for parameters pf=0.1, sp=100, lp=0500
Saving plots for parameters pf=0.1, sp=100, lp=1000
Saving plots for parameters pf=0.1, sp=200, lp=0005
Saving plots

In [331]:
flock_results = 

,pf,sp,lp,polar_order_avg,polar_order_avg_std
0,0.1,005,0005,0.001157,0.009948
1,0.1,005,0010,0.002199,0.010511
2,0.1,005,0020,0.003161,0.010176
3,0.1,005,0030,-0.000044,0.010834
4,0.1,005,0040,0.003365,0.009682
...,...,...,...,...,...
187,0.1,200,0150,0.804123,0.021889
188,0.1,200,0200,0.840441,0.013428
189,0.1,200,0300,0.858153,0.015149
190,0.1,200,0500,0.871151,0.018798


In [ ]:
#createAnalysisPlots(df)

In [ ]:
#createLocalPDFPlot1D(df)

In [ ]:
#createStructurePlots(df)

In [ ]:
sps,means = createPolarOrderAvgPlots(df)

In [ ]:
plt.rc('text', usetex=True)
createOrientationCorrPlots(df)

In [ ]:
createStrutureRadialAvgPlots(df)

In [ ]:
createSnapshotPlots(df,'.')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
import matplotlib.patches as mpatches

sps = [15.0, 20.0, 25.0, 50.0, 80.0, 100.0]
#sps = [15.0, 25.0, 50.0, 100.0]
lps = [20.0, 50.0, 100.0]
X=[]

# Labeling the training data for supervised learning
y=[0,0,0, 0,0,1, 0,1,2, 1,2,2, 3,3,3, 3,3,3, 2,2] # pf0.2
#y=[0,0,0, 0,0,1, 1,1,2, 1,2,2, 3,1,1, 3,1,1, 2,2] # pf0.2

#y=[0,1,1, 2,1,1, 2,1,1, 3,2,2]

#y=[0,0,0, 0,1,1, 1,2,2, 3,1,1]
# Build the dataset of training points
for s in sps:
    for l in lps:
        X.append([s,l])
        
X.append([30,100])
X.append([80, 20])

X = np.array(X)
y = np.array(y)

# Fit the data using a gaussian kernel
# The variance and magnitude of the kernel was found by
# trial and error to generate sensible phase boundaries.
kernel = 60*RBF(25)
clf = SVC(kernel=kernel, gamma='auto', tol=1e-6,
          probability=True, max_iter=1e8, decision_function_shape='ovr').fit(X,y)

# create a mesh for the colorplot
x_min, x_max = X[:, 0].min() - 10, X[:, 0].max() + 10
y_min, y_max = X[:, 1].min() - 10, X[:, 1].max() + 10

h = .1  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
alpha = 1
red = [1,0,0,alpha] #pf0.2
lime = [0,1,0,alpha] #pf0.1
blue = [0,0,1,alpha] #pf0.4
cyan = [0,1,1,alpha] #pf0.04
magenta = [1,0,1,alpha]

red_patch = mpatches.Patch(color=red[:3],
                           label='Active isotropic')
lime_patch = mpatches.Patch(color=lime[:3],
                            label='Flocking')
blue_patch = mpatches.Patch(color=blue[:3],
                            label='Polar band')
cyan_patch = mpatches.Patch(color=cyan[:3],
                            label='Spooling')
magenta_patch = mpatches.Patch(color=magenta[:3],
                               label='Turbulent')
colors = np.array([red, lime, blue, cyan, magenta])

plt.figure(figsize=(8, 8))

# Plot the predicted probabilities. For that, we will 
# assign a color to each point in the mesh
# [x_min, m_max]x[y_min, y_max].

#Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
#Z = np.array([colors[i] for i in Z])

# Put the result into a color plot
#Z = Z.reshape((xx.shape[0], xx.shape[1], 4))

#plt.imshow(Z, extent=(x_min, x_max, y_min, y_max),
           #origin="lower")

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, m_max]x[y_min, y_max].
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.contour(xx, yy, Z, colors='k')#, cmap=plt.cm.Paired)

# Plot the training points
plt.scatter(X[:, 0], X[:, 1], s=80,
            #c=[list(colors[i][:3]) for i in y], 
            c=blue,
            edgecolors=(0, 0, 0))
plt.ylabel(r'$L_p/L$', fontsize=20)
plt.xlabel(r'$U_{max}/k_B T$', fontsize=20)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())

plt.title(r"Phase diagram, $\phi = 0.2$", fontsize=25)

#plt.legend(handles=[red_patch, lime_patch, blue_patch,
#                    cyan_patch, magenta_patch], 
#           bbox_to_anchor=(1.01, 1))
plt.tight_layout()
plt.savefig('pf0.2_phase_diagram.png', dpi=100)
plt.show()

In [ ]:
def createStructurePlots(df, saveDirName=".", colorMap=cm.viridis):
    """Generates a grid of structure factors from time averaged 
    density FFTs.
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'structure'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    fig.tight_layout(pad=15,h_pad=0,w_pad=0)
    fig.subplots_adjust(right=0.8)
    pf = df.iloc[0].pf
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname = df.iloc[i][analyze]
        n = sum(1 for line in open(fname)) #number of records in file
        skip = list(range(n))
        del skip[::10]
        if (df.iloc[i].sp != 80):
            data = pd.read_csv(fname, delim_whitespace=True, header=None, skiprows=skip)
            data = data.iloc[::10,:]
        else:
            data = pd.read_csv(fname, delim_whitespace=True, header=None, skiprows=skip)
        data = data*1e4
        data = np.fft.fftshift(data)
        min_data = data.min().min()
        if (min_data == 0):
            min_data = 1e-16
        max_data = data.max().max()
        log_norm = LogNorm(vmin=min_data, vmax=max_data)
        cbar_ticks = [10**k for k in 
                      range(int(np.floor(np.log10(min_data))),
                            1 + int(np.ceil(np.log10(max_data))))]
        im = a.pcolormesh(data, norm=log_norm, cmap=cm.viridis)
        #fig.colorbar(im, cax=cbar_ax, ticks=cbar_ticks)
        a.set_xlabel('')
        a.tick_params(length=0,width=0,labelsize=20)
        a.set_xticklabels([])
        a.set_yticklabels([])
    fig.colorbar(im, cax=cbar_ax, ticks=cbar_ticks)
    for irow in range(fig_y): 
        a=ax[irow][0]
        a.set_ylabel(r'$k_y$',fontsize=30)
        a.locator_params(axis='y',nbins=11)
        a.tick_params(axis='y',length=5,width=2.5,labelsize=20)
        ticks=a.get_yticks()
        a.set_yticklabels(['{0:2.2f}'.format(2*np.pi*(x-500)/1000) 
                           for x in np.linspace(0,1000,len(ticks))])
    for icol in range(fig_x): 
        a=ax[fig_y-1][icol]
        a.set_xlabel(r'$k_x$',fontsize=30)
        a.locator_params(axis='x',nbins=11)
        a.tick_params(axis='x',length=5,width=2.5,labelsize=20)
        ticks=a.get_xticks()
        a.set_xticklabels(['{0:2.2f}'.format(2*np.pi*(x-500)/1000) 
                           for x in np.linspace(0,1000,len(ticks))])
    cbar_ax.tick_params(length=10,width=5,labelsize=30)
    fig.text(0.45, 0.01,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center',fontsize=70)
    fig.text(0.0, 0.5, r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical',fontsize=70)
    fig.suptitle("Structure Factor, "+str(100*pf)+"% pf",
                 fontsize=70, y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_structure_factor_plots.png",dpi=300)


In [ ]:
#createOverlapPlots(df)

In [ ]:
createStructurePlots(df)

In [ ]:
#createFileList('structure','pf0.2/condensed_results')

In [ ]:
L=[1,2,3,4]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D

sps = [15.0, 20.0, 25.0, 50.0, 80.0, 100.0]
lps = [20.0, 50.0, 100.0]
X=[]

# Labeling the training data for supervised learning
y=[0,0,0, 0,0,1, 1,1,2, 1,2,2, 3,4,4, 3,4,4, 2,2] 
# Build the dataset of training points
for s in sps:
    for l in lps:
        X.append([s,l])
        
X.append([30,100])
X.append([60,100])

X = np.array(X)
y = np.array(y)

# Fit the data using a gaussian kernel
# The variance and magnitude of the kernel was found by
# trial and error to generate sensible phase boundaries.
kernel = 50*RBF(20)
clf = SVC(kernel=kernel, gamma='auto', tol=1e-6,
          probability=True, max_iter=1e8).fit(X,y)

# create a mesh for the colorplot
x_min, x_max = X[:, 0].min() - 10, X[:, 0].max() + 10
y_min, y_max = X[:, 1].min() - 10, X[:, 1].max() + 10

h = .1  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
alpha = 0.4
red = [1,0,0,alpha]
lime = [0,1,0,alpha]
blue = [0,0,1,alpha]
cyan = [0,1,1,alpha]
magenta = [1,0,1,alpha]

red_patch = mpatches.Patch(color=red[:3],
                           label='Active isotropic')
lime_patch = mpatches.Patch(color=lime[:3],
                            label='Flocking')
blue_patch = mpatches.Patch(color=blue[:3],
                            label='Polar band')
cyan_patch = mpatches.Patch(color=cyan[:3],
                            label='Spooling')
magenta_patch = mpatches.Patch(color=magenta[:3],
                               label='Turbulent')
colors = np.array([red, lime, blue, cyan, magenta])

#plt.figure(figsize=(8, 8))

# Plot the predicted probabilities. For that, we will 
# assign a color to each point in the mesh
# [x_min, m_max]x[y_min, y_max].

Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = np.array([colors[i] for i in Z])

# Put the result into a color plot
Z = Z.reshape((xx.shape[0], xx.shape[1], 4))
fig = plt.figure()
ax = fig.gca(projection='3d')

ax.imshow(Z, origin="lower",extent=(x_min, x_max, y_min, y_max))

# Plot the training points
ax.scatter(X[:, 0], X[:, 1], zs=0.2, zdir='z', s=80,
            c=[list(colors[i][:3]) for i in y], 
            edgecolors=(0, 0, 0))

In [ ]:
plt.show()

In [ ]:
fig, a= plt.subplots()
pf = df.iloc[0].pf
analyze = 'structure'
#a = ax[irow][icol]
fname = df.iloc[0][analyze]
n = sum(1 for line in open(fname)) #number of records in file
skip = list(range(n))
#del skip[::10]
data = pd.read_csv(fname, delim_whitespace=True, header=None)#, skiprows=skip)
#data = data.iloc[::10,:]
data = data*1e4
data = np.fft.fftshift(data)
min_data = data.min().min()
if (min_data == 0):
    min_data = 1e-16
max_data = data.max().max()
log_norm = LogNorm(vmin=min_data, vmax=max_data)
cbar_ticks = [10**k for k in 
              range(int(np.floor(np.log10(min_data))),
                    1 + int(np.ceil(np.log10(max_data))))]
im = a.pcolormesh(data, norm=log_norm, cmap=cm.viridis)
#fig.colorbar(im, cax=cbar_ax, ticks=cbar_ticks)
a.set_xlabel('')
a.tick_params(length=0,width=0,labelsize=20)


In [ ]:
def createPolarOrderAvgPlots(df,saveDirName="."):
    """Generates a grids of plots displaying time series of the average
    local polar order.
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'polar_order_avg'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]   

    # First, we're going to plot nematic and polar global order params
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    pf = df.iloc[0].pf
    means = np.zeros((fig_y, fig_x))
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        goDF=pd.read_csv(fname, skiprows=1, delim_whitespace=True)
        goDF.plot(x="time", y="avg_polar_order", color='blue',
                  linewidth=3, ax=a)
        late_time = int(2/3 * len(goDF['avg_polar_order']))
        late_sim_mean = goDF['avg_polar_order'].iloc[late_time:].mean()
        means[irow][icol] = late_sim_mean
        a.set_xlabel('')
        a.tick_params(length=5, width=2.5, labelsize=20)
        a.grid(True, linestyle='dashed')
    for irow in range(fig_y): 
        ax[irow][0].set_ylabel(r'$\langle$ Local polar order $\rangle$',
                               fontsize=30)
    for icol in range(fig_x): 
        ax[fig_y-1][icol].set_xlabel('Time', fontsize=30)
    fig.tight_layout(pad=15, h_pad=0, w_pad=0)
    fig.text(0.5, 0.05,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center', fontsize=70)
    fig.text(0.01, 0.5,
             r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical', fontsize=70)
    fig.suptitle("Average Local Polar Order, "+str(100*pf)+"% pf",
                 fontsize=70, y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_polar_order_avg_plots.png", dpi=300)
    
    # Now create aggregate data plot for the late sim-time means
    fig, ax = plt.subplots(1,1)
    for i in range(means.shape[0]):
        ax.plot(sps, means[i], '-o')
    ax.grid(True, linestyle='dashed')
    ax.legend([r'$L_p/L=$'+str(i) for i in lps[::-1]],fontsize=15)
    ax.set_xlabel(r'$U_{max}/k_B T$',fontsize=15)
    ax.set_ylabel(r'$\langle p_i \rangle$',fontsize=15)
    ax.set_title("Average local polar order")
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_polar_order_avg_aggregate.png",dpi=300)
    plt.close(fig)

    # First, we're going to plot nematic and polar global order params
    fig,ax,fig_x,fig_y,sps_cn,lps = GetAnalysisGridPlotHandles(df,analyze)
    pf = df.iloc[0].pf
    means_cn = np.zeros((fig_y, fig_x))
    for i in df.index:
        icol=np.where(sps_cn==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        goDF=pd.read_csv(fname, skiprows=1, delim_whitespace=True)
        print(goDF.head())
        goDF.plot(x="time", y="avg_contact_number", color='blue',
                  linewidth=3, ax=a)
        late_time = int(2/3 * len(goDF['avg_contact_number']))
        late_sim_mean = goDF['avg_contact_number'].iloc[late_time:].mean()
        means_cn[irow][icol] = late_sim_mean
        a.set_xlabel('')
        a.tick_params(length=5, width=2.5, labelsize=20)
        a.grid(True, linestyle='dashed')
    for irow in range(fig_y): 
        ax[irow][0].set_ylabel(r'$\langle$ Average contact number $\rangle$',
                               fontsize=30)
    for icol in range(fig_x): 
        ax[fig_y-1][icol].set_xlabel('Time', fontsize=30)
    fig.tight_layout(pad=15, h_pad=0, w_pad=0)
    fig.text(0.5, 0.05,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center', fontsize=70)
    fig.text(0.01, 0.5,
             r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical', fontsize=70)
    fig.suptitle("Average Contact Number, "+str(100*pf)+"% pf",
                 fontsize=70, y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_contact_number_avg_plots.png", dpi=300)
    
    # Now create aggregate data plot for the late sim-time means
    fig, ax = plt.subplots(1,1)
    for i in range(means_cn.shape[0]):
        ax.plot(sps_cn, means_cn[i], '-o')
    ax.grid(True, linestyle='dashed')
    ax.legend([r'$L_p/L=$'+str(i) for i in lps[::-1]],fontsize=15)
    ax.set_xlabel(r'$U_{max}/k_B T$',fontsize=15)
    ax.set_ylabel(r'$\langle p_i \rangle$',fontsize=15)
    ax.set_title("Average contact number")
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"_contact_number_avg_aggregate.png",dpi=300)
    return sps,means,sps_cn,means_cn

In [ ]:
createPolarOrderAvgPlots(df, 'pf0.1/l050/plots')

In [ ]:
fname = df['flock'].iloc[0]
#f = open('soft_pf0.1_sp050_lp100_condensed_filament.flock', 'r')
f = open(fname, 'r')
f.readline() # burn first line of file
# get parameter names and values
param_names = f.readline().strip('\n').split(' ')
param_vals = list(map(float, f.readline().split(' ')))
f.close()
params = {i:j for i,j in zip(param_names, param_vals)}

L = 50
sysL = 1000
fil_vol = L+0.25*np.pi
num_filaments = int(0.1*sysL**2/fil_vol)

def smooth(x, N):
    k = x.rolling(window=N).mean().iloc[N-1:].reset_index(drop=True)
    k.name = x.name + "_smooth"
    return k

flock_df = pd.read_csv(fname, delim_whitespace=True, skiprows=3)
smooth_N = 100
flock_df['flux_net'] = flock_df['n_joined'] - flock_df['n_left']
flock_df['time_smooth'] = flock_df.time.iloc[smooth_N-1:].reset_index(drop=True)
flock_df['n_joined_smooth'] = smooth(flock_df.n_joined, smooth_N)
flock_df['n_left_smooth'] = smooth(flock_df.n_left, smooth_N)
flock_df['flux_net_smooth'] = smooth(flock_df.flux_net, smooth_N)
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15, 5))
ax0.plot(flock_df.time[:-smooth_N+1], smooth(flock_df.n_flocking, smooth_N))
ax0.plot(flock_df.time[:-smooth_N+1], smooth(flock_df.n_interior, smooth_N))
ax0.plot(flock_df.time[:-smooth_N+1], smooth(flock_df.n_exterior, smooth_N))
ax0.hlines(num_filaments, flock_df.time[0], flock_df.time.iloc[-1], colors='k', linestyles='dashed')
ax0.legend(['num flocking', 'num interior', 'num exterior', 'num filaments'],fontsize=15)
ax0.set_xlabel('time',fontsize=15)
ax0.set_ylabel('filament number',fontsize=15)

ax1.plot(flock_df.time_smooth, flock_df.n_joined_smooth)
ax1.plot(flock_df.time_smooth, flock_df.n_left_smooth)
ax1.plot(flock_df.time_smooth, flock_df.flux_net_smooth)
ax1.hlines(0, flock_df.time_smooth.iloc[0], flock_df.time_smooth.iloc[-smooth_N], colors='k', linestyles='dashed')
ax1.legend(['flux in', 'flux out', 'net flux'], fontsize=15)
ax1.set_xlabel('time', fontsize=15)
ax1.set_ylabel('filament flux', fontsize=15)
#ax1.set_xlim(0,5)
plt.show()

In [ ]:
def createFlockPlots(df,saveDirName="."):
    """Generates a grids of plots displaying time series of the average
    local polar order.
    """
    assert isinstance(saveDirName,str), "'dirName' must be a string!"
    analyze = 'flock'
    if saveDirName[-1]=='/': 
        saveDirName=saveDirName[:-1]   

    # First, we're going to plot nematic and polar global order params
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    pf = df.iloc[0].pf
    means = np.zeros((fig_y, fig_x))
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        flock_df=pd.read_csv(fname, skiprows=3, delim_whitespace=True)
        smooth_N = 100
        a.plot(flock_df.time[:-smooth_N+1], smooth(flock_df.n_flocking, smooth_N))
        a.plot(flock_df.time[:-smooth_N+1], smooth(flock_df.n_interior, smooth_N))
        a.plot(flock_df.time[:-smooth_N+1], smooth(flock_df.n_exterior, smooth_N))
        a.hlines(num_filaments, flock_df.time[0], flock_df.time.iloc[-1], colors='k', linestyles='dashed')
        a.legend(['num flocking', 'num interior', 'num exterior', 'num filaments'],fontsize=15)
        a.set_xlabel('time',fontsize=15)
        a.set_ylabel('filament number',fontsize=15)
        late_time = int(2/3 * len(flock_df.n_flocking))
        late_sim_mean = flock_df.n_flocking.iloc[late_time:].mean()
        means[irow][icol] = late_sim_mean
        #a.set_xlabel('')
        a.tick_params(length=5, width=2.5, labelsize=20)
        a.grid(True, linestyle='dashed')
    for irow in range(fig_y): 
        ax[irow][0].set_ylabel(r'$\langle$ Filament nunmber $\rangle$',
                               fontsize=30)
    for icol in range(fig_x): 
        ax[fig_y-1][icol].set_xlabel('Time', fontsize=30)
    fig.tight_layout(pad=15, h_pad=0, w_pad=0)
    fig.text(0.5, 0.05,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center', fontsize=70)
    fig.text(0.01, 0.5,
             r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical', fontsize=70)
    fig.suptitle("Average flocking filaments, "+str(100*pf)+"% pf",
                 fontsize=70, y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"flock_number_plots.png", dpi=300)

    plt.close(fig)
    fig,ax,fig_x,fig_y,sps,lps = GetAnalysisGridPlotHandles(df,analyze)
    for i in df.index:
        icol=np.where(sps==df.iloc[i].sp)[0][0]
        irow=fig_y-np.where(lps==df.iloc[i].lp)[0][0]-1
        a = ax[irow][icol]
        fname=df.iloc[i][analyze]
        flock_df=pd.read_csv(fname, skiprows=3, delim_whitespace=True)
        smooth_N = 100
        flock_df['flux_net'] = flock_df['n_joined'] - flock_df['n_left']
        flock_df['time_smooth'] = flock_df.time.iloc[smooth_N-1:].reset_index(drop=True)
        flock_df['n_joined_smooth'] = smooth(flock_df.n_joined, smooth_N)
        flock_df['n_left_smooth'] = smooth(flock_df.n_left, smooth_N)
        flock_df['flux_net_smooth'] = smooth(flock_df.flux_net, smooth_N)
        a.plot(flock_df.time_smooth, flock_df.n_joined_smooth)
        a.plot(flock_df.time_smooth, flock_df.n_left_smooth)
        a.plot(flock_df.time_smooth, flock_df.flux_net_smooth)
        a.hlines(0, flock_df.time_smooth.iloc[0], flock_df.time_smooth.iloc[-smooth_N], colors='k', linestyles='dashed')
        a.legend(['flux in', 'flux out', 'net flux'], fontsize=15)
        a.set_xlabel('time', fontsize=15)
        a.set_ylabel('filament flux', fontsize=15)

        a.tick_params(length=5, width=2.5, labelsize=20)
        a.grid(True, linestyle='dashed')
    for irow in range(fig_y): 
        ax[irow][0].set_ylabel(r'$\langle$ Filament flux $\rangle$',
                               fontsize=30)
    for icol in range(fig_x): 
        ax[fig_y-1][icol].set_xlabel('Time', fontsize=30)
    fig.tight_layout(pad=15, h_pad=0, w_pad=0)
    fig.text(0.5, 0.05,
             r'$\Longrightarrow U_{max}/k_B T \Longrightarrow$',
             ha='center', fontsize=70)
    fig.text(0.01, 0.5,
             r'$\Longrightarrow L_p/L \Longrightarrow$',
             va='center', rotation='vertical', fontsize=70)
    fig.suptitle("Average filament flux, "+str(100*pf)+"% pf",
                 fontsize=70, y=0.95)
    fig.savefig(saveDirName+"/pf"+str(pf)
                +"flock_flux_plots.png", dpi=300)

In [ ]:
createFlockPlots(df,"pf0.1/plots/")

In [38]:
df

,pf,sp,lp,global_order,polar_order,polar_order_avg
0,0.1,5,5,./order_params/reload1/activeff_pf0.1_sp005_lp...,./order_params/reload1/activeff_pf0.1_sp005_lp...,./order_params/reload1/activeff_pf0.1_sp005_lp...
1,0.1,5,10,./order_params/reload1/activeff_pf0.1_sp005_lp...,./order_params/reload1/activeff_pf0.1_sp005_lp...,./order_params/reload1/activeff_pf0.1_sp005_lp...
2,0.1,5,20,./order_params/reload1/activeff_pf0.1_sp005_lp...,./order_params/reload1/activeff_pf0.1_sp005_lp...,./order_params/reload1/activeff_pf0.1_sp005_lp...
3,0.1,5,30,./order_params/reload1/activeff_pf0.1_sp005_lp...,./order_params/reload1/activeff_pf0.1_sp005_lp...,./order_params/reload1/activeff_pf0.1_sp005_lp...
4,0.1,5,40,./order_params/reload1/activeff_pf0.1_sp005_lp...,./order_params/reload1/activeff_pf0.1_sp005_lp...,./order_params/reload1/activeff_pf0.1_sp005_lp...
...,...,...,...,...,...,...
187,0.1,200,150,./order_params/reload1/activeff_pf0.1_sp200_lp...,./order_params/reload1/activeff_pf0.1_sp200_lp...,./order_params/reload1/activeff_pf0.1_sp200_lp...
188,0.1,200,200,./order_params/reload1/activeff_pf0.1_sp200_lp...,./order_params/reload1/activeff_pf0.1_sp200_lp...,./order_params/reload1/activeff_pf0.1_sp200_lp...
189,0.1,200,300,./order_params/reload1/activeff_pf0.1_sp200_lp...,./order_params/reload1/activeff_pf0.1_sp200_lp...,./order_params/reload1/activeff_pf0.1_sp200_lp...
190,0.1,200,500,./order_params/reload1/activeff_pf0.1_sp200_lp...,./order_params/reload1/activeff_pf0.1_sp200_lp...,./order_params/reload1/activeff_pf0.1_sp200_lp...


In [122]:
fname = df.iloc[0].global_order
match = re.search('pf0?.?[0-9]+_sp[0-9]+_lp[0-9]+', fname)
match.group()

'pf0.1_sp005_lp0005'

In [288]:
def get_flock_df(fname):
    df = pd.read_csv(fname, header=3, low_memory=False, delim_whitespace=True)
    header = pd.read_csv(fname, header=1, nrows=1, delim_whitespace=True)
    filcols = [col for col in df.columns if col[:3] == 'fil']
    flockcols = [col for col in df.columns if col[:3] != 'fil']

    flock_global = df[flockcols].dropna()
    flock_global = flock_global[['n_flocking', 'n_exterior', 'n_interior']]

    flockstates = df[filcols].dropna().values
    n_filaments = flockstates.shape[1]

    flockstates[flockstates == 2] = 3
    diffs = pd.DataFrame(np.diff(flockstates, axis=0))

    freqs = ['f_not_ext', 'f_not_int', 'f_ext_int','f_ext_not', 'f_int_ext', 'f_int_not']
    flock_state = ['n_not', 'n_ext', 'n_int']
    change_state = [1, 3, 2, -1, -2, -3]
    #change_state = [-3, -2, -1, 1, 2, 3]
    df = pd.DataFrame(columns=freqs + flock_state)
    df['n_not'] = n_filaments - flock_global['n_flocking']
    df['n_ext'] = flock_global['n_exterior']
    df['n_int'] = flock_global['n_interior']
    for freq, state in zip(freqs, change_state):
        if freq[2:5] == 'int':
            denom = df['n_int']
        elif freq[2:5] == 'ext':
            denom = df['n_ext']
        elif freq[2:5] == 'not':
            denom = df['n_not']
        else:
            raise ValueError("Unexpected frequency")
        df[freq] = diffs[diffs==state].count(axis=1) / denom
    df = df.iloc[1:-1, :].fillna(0)
    #flock_global = flock_global.iloc[1:, :].rolling(20).mean().dropna()
    step = 0.5 * header['nspec'][0] * header['delta'][0]
    time = np.linspace(0, flock_global.shape[0]*step, flock_global.shape[0])
    df['time'] = df.index * step
    for state in flock_state:
        df[state] = df[state] / n_filaments
    df['n_tot'] = 1 - df['n_not']
    return df

def plot_flock_state(df, display_string, save_string):
    # Rolling time average with window = 20, 1 tau for nspec = 1000, delta = 0.0001
    df = df.rolling(20).mean().dropna()
    # Now plot them
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].set_title('Fraction of flocking filaments')
    ax[0].plot(df['time'], df['n_tot'], label='total')
    ax[0].plot(df['time'], df['n_ext'], label='exterior')
    ax[0].plot(df['time'], df['n_int'], label='interior')
    ax[0].set_xlabel('Time')
    ax[0].set_ylabel('Filament fraction')
    ax[0].legend(loc='best')
    freq_cols = [col for col in df.columns if col[:2] == 'f_']
    df[['time'] + freq_cols].plot(x='time', ax=ax[1], title='Normalized flock switching rates')
    ax[1].set_ylabel('Frequency')
    ax[1].set_xlabel('Time')
    fig.suptitle("Flock dynamics: "+display_string)
    fig.savefig(save_string)
    print("Saving plots for parameters", display_string)
    plt.close(fig)


def make_flock_plots(df, saveDirName=".", params=['pf', 'sp', 'lp']):
    """TODO"""
    assert isinstance(saveDirName, str), "'dirName' must be a string!"
    analyze = 'flock'
    if not os.path.exists(saveDirName):
        print("Save directory not found:", saveDirName)
        var = input("Create it? (y/N) ")
        if (var == 'y' or var == 'Y'):
            os.mkdir(saveDirName)
        else:
            raise ValueError("Save directory not found", saveDirName)
    gby = df.groupby(params)
    for values, group in gby:
        #fig, ax = plt.subplots(1, 2, figsize=(12, 6))
        flock_df = None
        for file in group[analyze].sort_values():
            if flock_df is not None: 
                flock_df = flock_df.append(get_flock_df(file), ignore_index=True)
            else:
                flock_df = get_flock_df(file)
        flock_df['time'] = flock_df.index * flock_df['time'].iloc[0]
        param_values = [i for pair in zip(params, values) for i in pair]
        string_values = str.join('_', ['{}{}' for i in range(len(params))])
        string_values = string_values.format(*param_values)
        display_values = str.join(', ', ['{}={}' for i in range(len(params))])
        display_values = display_values.format(*param_values)
        plot_flock_state(flock_df, display_values,
                         os.path.join(saveDirName, string_values + "_flock.png"))

In [289]:
df = initializeDataFrame('order_params/',
                    params=['pf', 'sp', 'lp'],
                    analyses=['global_order',
                              'polar_order',
                              'polar_order_avg',
                              'flock'])

In [290]:
make_flock_plots(df, saveDirName='order_params/plots/', params=['pf', 'sp', 'lp'])

Saving plots for parameters pf=0.1, sp=005, lp=0005
Saving plots for parameters pf=0.1, sp=005, lp=0010
Saving plots for parameters pf=0.1, sp=005, lp=0020
Saving plots for parameters pf=0.1, sp=005, lp=0030
Saving plots for parameters pf=0.1, sp=005, lp=0040
Saving plots for parameters pf=0.1, sp=005, lp=0050
Saving plots for parameters pf=0.1, sp=005, lp=0060
Saving plots for parameters pf=0.1, sp=005, lp=0070
Saving plots for parameters pf=0.1, sp=005, lp=0080
Saving plots for parameters pf=0.1, sp=005, lp=0090
Saving plots for parameters pf=0.1, sp=005, lp=0100
Saving plots for parameters pf=0.1, sp=005, lp=0150
Saving plots for parameters pf=0.1, sp=005, lp=0200
Saving plots for parameters pf=0.1, sp=005, lp=0300
Saving plots for parameters pf=0.1, sp=005, lp=0500
Saving plots for parameters pf=0.1, sp=005, lp=1000
Saving plots for parameters pf=0.1, sp=010, lp=0005
Saving plots for parameters pf=0.1, sp=010, lp=0010
Saving plots for parameters pf=0.1, sp=010, lp=0020
Saving plots

Saving plots for parameters pf=0.1, sp=090, lp=0500
Saving plots for parameters pf=0.1, sp=090, lp=1000
Saving plots for parameters pf=0.1, sp=100, lp=0005
Saving plots for parameters pf=0.1, sp=100, lp=0010
Saving plots for parameters pf=0.1, sp=100, lp=0020
Saving plots for parameters pf=0.1, sp=100, lp=0030
Saving plots for parameters pf=0.1, sp=100, lp=0040
Saving plots for parameters pf=0.1, sp=100, lp=0050
Saving plots for parameters pf=0.1, sp=100, lp=0060
Saving plots for parameters pf=0.1, sp=100, lp=0070
Saving plots for parameters pf=0.1, sp=100, lp=0080
Saving plots for parameters pf=0.1, sp=100, lp=0090
Saving plots for parameters pf=0.1, sp=100, lp=0100
Saving plots for parameters pf=0.1, sp=100, lp=0150
Saving plots for parameters pf=0.1, sp=100, lp=0200
Saving plots for parameters pf=0.1, sp=100, lp=0300
Saving plots for parameters pf=0.1, sp=100, lp=0500
Saving plots for parameters pf=0.1, sp=100, lp=1000
Saving plots for parameters pf=0.1, sp=200, lp=0005
Saving plots

In [292]:
df

,pf,sp,lp,reload,global_order,polar_order,polar_order_avg,flock
0,0.1,005,0005,001,order_params/activeff_pf0.1_sp005_lp0005_reloa...,order_params/activeff_pf0.1_sp005_lp0005_reloa...,order_params/activeff_pf0.1_sp005_lp0005_reloa...,order_params/activeff_pf0.1_sp005_lp0005_reloa...
1,0.1,005,0005,002,order_params/activeff_pf0.1_sp005_lp0005_reloa...,order_params/activeff_pf0.1_sp005_lp0005_reloa...,order_params/activeff_pf0.1_sp005_lp0005_reloa...,order_params/activeff_pf0.1_sp005_lp0005_reloa...
2,0.1,005,0010,001,order_params/activeff_pf0.1_sp005_lp0010_reloa...,order_params/activeff_pf0.1_sp005_lp0010_reloa...,order_params/activeff_pf0.1_sp005_lp0010_reloa...,order_params/activeff_pf0.1_sp005_lp0010_reloa...
3,0.1,005,0010,002,order_params/activeff_pf0.1_sp005_lp0010_reloa...,order_params/activeff_pf0.1_sp005_lp0010_reloa...,order_params/activeff_pf0.1_sp005_lp0010_reloa...,order_params/activeff_pf0.1_sp005_lp0010_reloa...
4,0.1,005,0020,001,order_params/activeff_pf0.1_sp005_lp0020_reloa...,order_params/activeff_pf0.1_sp005_lp0020_reloa...,order_params/activeff_pf0.1_sp005_lp0020_reloa...,order_params/activeff_pf0.1_sp005_lp0020_reloa...
...,...,...,...,...,...,...,...,...
379,0.1,200,0300,002,order_params/activeff_pf0.1_sp200_lp0300_reloa...,order_params/activeff_pf0.1_sp200_lp0300_reloa...,order_params/activeff_pf0.1_sp200_lp0300_reloa...,order_params/activeff_pf0.1_sp200_lp0300_reloa...
380,0.1,200,0500,001,order_params/activeff_pf0.1_sp200_lp0500_reloa...,order_params/activeff_pf0.1_sp200_lp0500_reloa...,order_params/activeff_pf0.1_sp200_lp0500_reloa...,order_params/activeff_pf0.1_sp200_lp0500_reloa...
381,0.1,200,0500,002,order_params/activeff_pf0.1_sp200_lp0500_reloa...,order_params/activeff_pf0.1_sp200_lp0500_reloa...,order_params/activeff_pf0.1_sp200_lp0500_reloa...,order_params/activeff_pf0.1_sp200_lp0500_reloa...
382,0.1,200,1000,001,order_params/activeff_pf0.1_sp200_lp1000_reloa...,order_params/activeff_pf0.1_sp200_lp1000_reloa...,order_params/activeff_pf0.1_sp200_lp1000_reloa...,order_params/activeff_pf0.1_sp200_lp1000_reloa...


In [309]:
k = pd.DataFrame(columns=['a', 'b', 'c'])

In [316]:
params = ['a', 'b', 'c']
values = (1, 2, 3)
analyze = ['d', 'e']
results = (4, 5)
row = {key:value for key, value in list(zip(params, values)) + list(zip(analyze, results))}
#row2 = {key:value for key, value in zip(analyze, results)}

In [317]:
row

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5}

In [319]:
int(4.9)

4